In [5]:
# Install compatible Chrome version
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

# Get Chrome version
!google-chrome --version


--2025-06-09 08:36:17--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 172.217.7.46, 2607:f8b0:4025:803::200e
Connecting to dl.google.com (dl.google.com)|172.217.7.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117334784 (112M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 111.90M  25.5MB/s    in 5.2s    

2025-06-09 08:36:22 (21.4 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [117334784/117334784]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 126568 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (137.0.7151.68-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Package libvulkan1 is 

In [6]:
!wget -O /tmp/chromedriver.zip https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/137.0.7151.68/linux64/chromedriver-linux64.zip
!unzip /tmp/chromedriver.zip -d /usr/bin/
!mv /usr/bin/chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver


--2025-06-09 08:37:32--  https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/137.0.7151.68/linux64/chromedriver-linux64.zip
Resolving edgedl.me.gvt1.com (edgedl.me.gvt1.com)... 34.104.35.123, 2600:1900:4110:86f::
Connecting to edgedl.me.gvt1.com (edgedl.me.gvt1.com)|34.104.35.123|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.68/linux64/chromedriver-linux64.zip [following]
--2025-06-09 08:37:32--  https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.68/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.0.91, 172.217.7.59, 172.217.12.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.0.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9784384 (9.3M) [application/zip]
Saving to: ‘/tmp/chromedriver.zip’

/tmp/chromedriver.z 100%[===================>]   9.33M  33.9M

In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from tqdm import tqdm

# Setup Chrome options
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Start the browser
driver = webdriver.Chrome(service=Service("/usr/bin/chromedriver"), options=options)

# Base URL
base_url = "https://kando.tech/investors/all?page="

all_data = []
total_pages = 634  # page=0 to page=633

# Loop over all pages
for page in tqdm(range(0, total_pages)):
    url = base_url + str(page)
    driver.get(url)
    time.sleep(2)  # let page load

    rows = driver.find_elements(By.CSS_SELECTOR, "tr")

    for row in rows:
        try:
            cols = row.find_elements(By.TAG_NAME, "td")
            if len(cols) < 5:
                continue

            # Try to find <a>; fallback to plain text if missing
            try:
                name_elem = cols[1].find_element(By.TAG_NAME, "a")
                name = name_elem.text.strip()
                link = name_elem.get_attribute("href")
            except:
                name = cols[1].text
                link = None  # or ""

            country = cols[2].text.strip()
            profile = cols[3].text.strip()
            deals = cols[4].text.strip()

            all_data.append({
                "Name": name,
                "Country": country,
                "Profile": profile,
                "Deals": deals,
                "Link": link
            })

        except Exception as e:
            print(f"⚠️ Error on page {page}: {e}")

driver.quit()

# Save to CSV
df = pd.DataFrame(all_data)
df.to_csv("kando_all_investors.csv", index=False, quoting=csv.QUOTE_ALL)
print(f"✅ Scraped and saved {len(df)} investor records.")


100%|██████████| 634/634 [59:29<00:00,  5.63s/it]


✅ Scraped and saved 12533 investor records.
